# Setup process
1. Create database
2. Create bill table (id, chamber, number, year, url, rss, lastBuildDate)
3. create action table (title, description, pubDate)
   1. bills 1-to-n actions
4. For each bill, grab all actions from RSS

In [2]:
import sqlite3
# import pandas as pd
import feedparser
import re

db = "lege.db"

In [11]:
# NOTE Run to clear database entirely, including deleting tables
# con = sqlite3.connect(db)
# cur = con.cursor()
# cur.execute("DROP TABLE bills")
# cur.execute("DROP TABLE actions")
# con.commit()
# con.close()

In [12]:
# NOTE Run to set up database
# con = sqlite3.connect(db)
# cur = con.cursor()
# cur.execute("CREATE TABLE bills(id, chamber, number, year, url, rss, lastBuildDate)")
# cur.execute("CREATE TABLE actions(id, description, pubDate, scheduled, committee, date, time, location)")
# con.close()

In [3]:
# NOTE Run to clear the database without dropping tables
con = sqlite3.connect(db)
cur = con.cursor()
# cur.execute("DELETE FROM bills")
cur.execute("DELETE FROM actions")
con.commit()
con.close()

In [ ]:

# con = sqlite3.connect(db)
# bills = pd.read_sql("SELECT * from bills", con)
# actions = pd.read_sql("SELECT * from actions", con)
# con.close()

In [4]:
def userInterface():
    print("What would you like to do?")
    print("1. Add a Bill")
    print("2. View all Bills")
    print("3. View Timestamp of Last Run")
    print("4. Check for Updates")
    print()
    choice = input("Enter a number between 1 and 4: ")
    # TODO case switch 1-4
    # TODO take another action?

def addBill(db):
    id = input("Enter the bill number with a space, e.g. HD 1010: ")
    # TODO error handle bill format A{2}d{4} with regex

    year = input("Enter the bill year, e.g. 2024: ")
    # TODO error handle valid year d{4} > 2000 with regex

    # TODO check for duplicate ids
    chamber, number = id.split()
    chamber = chamber.upper()
    id = year + chamber + number

    # parse input to columns
    url = f"https://www.capitol.hawaii.gov/session/measure_indiv.aspx?billtype={chamber}&billnumber={number}&year={year}"
    rss = f"https://www.capitol.hawaii.gov/sessions/session{year}/rss/{chamber}{number}.xml"
    build = None

    # TODO validate row created well

    # add row to sql db
    row = [id, chamber, number, year, url, rss, build]
    con = sqlite3.connect(db)
    cur = con.cursor()
    cur.execute("INSERT INTO bills VALUES(?, ?, ?, ?, ?, ?, ?)", row)
    con.commit()
    con.close()
    # TODO return success/failure message

def getBill(db, id):
    con = sqlite3.connect(db)
    cur = con.cursor()
    with con:
        # FIXME: for actual implementation: chamber, number
        view = cur.execute("SELECT * FROM bills WHERE id=?", [id])
        print(view.fetchone())
    con.close()

def getBills(db):
    con = sqlite3.connect(db)
    cur = con.cursor()
    with con:
        # FIXME: for actual implementation: chamber, number
        view = cur.execute("SELECT * FROM bills")
        print(view.fetchall())
    con.close()

def getActions(db):
    con = sqlite3.connect(db)
    cur = con.cursor()
    with con:
        view = cur.execute("SELECT * FROM actions")
        print(view.fetchall())
    con.close()

def getHearings(db):
    con = sqlite3.connect(db)
    cur = con.cursor()
    with con:
        # FIXME: for actual implementation: chamber, number
        view = cur.execute("SELECT * FROM actions WHERE scheduled = TRUE")
        print(view.fetchall())
    con.close()

def viewTimestamp():
    # TODO create text file with timestamp of last run
    # if file not found, "file not found: never run" message
    pass

def checkAllUpdates(db):
    # TODO generator checkUpdate() for all bills...
    # iterate over all districts
    con = sqlite3.connect(db)
    cur = con.cursor()
    with con:
        bill_list = cur.execute("SELECT id FROM bills")
        bill_list = bill_list.fetchall()
        gen = (checkUpdate(id[0]) for id in bill_list)
        list(gen)
    con.close()

    # Update timestamp
    from time import strftime
    with open('timestamp.txt', 'w') as f:
        f.write(strftime("%d %b %Y %H:%M:%S"))
    
    # Return success/fail message
    print('Success: all bills updated from feed')

def checkUpdate(id):
    # connect to db
    con = sqlite3.connect(db)
    cur = con.cursor()
    
    # get rss feed
    feed = cur.execute("SELECT rss FROM bills WHERE id=?", [id])
    feed = feed.fetchone()[0]
    data = feedparser.parse(feed)
           
    # TODO compare lastupdated. If !=, grab latest item
    # print(data['feed']['published_parsed'])
    
    # get basic data from latest rss item
    description = data.entries[0].title
    pubDate = data.entries[0].published
    scheduled = "scheduled" in description # if scheduled, will grab hearing information
    
    # description template varies between Senate and House, so need to check chamber
    chamber = cur.execute("SELECT chamber FROM bills WHERE id=?", [id])
    chamber = chamber.fetchone()[0]
    senatecheck = chamber == 'SB'

    if scheduled and senatecheck: # if scheduled for a Senate hearing
        committee = re.findall("[^\\s]+(?= has)", description)
        date = re.findall("\\d{2}-\\d{02}-\\d{2}", description)
        time = re.findall("[^\\s]+(?=;)", description)
        location = re.findall("(?<=; ).*(?=\\.)", description)

        committee = committee[0]
        date = date[0]
        time = time[0]
        location = location[0]

        print(f"{id} has been scheduled for a hearing. Details: ")
        print(f"{committee} {date} {time}")
    
    elif scheduled: # if scheduled for a House hearing
        committee = re.findall("[^\\s]+(?= on)", description)
        date = re.findall("\\d{2}-\\d{02}-\\d{2}", description)
        time = re.findall("[^\\s]+(?= in)", description)
        location = re.findall("(?<= in ).*(?=\\.)", description)
        
        committee = committee[0]
        date = date[0]
        time = time[0]
        location = location[0]

        print(f"{id} has been scheduled for a hearing. Details: ")
        print(f"{committee} {date} {time}")

    else: # if not scheduled
        committee = None
        date = None
        time = None
        location = None

    # TODO validate row
        
    # save row to db
    # TODO handle duplicates: https://www.tutorialspoint.com/sql/sql-handling-duplicates.htm
    row = [id, description, pubDate, scheduled, committee, date, time, location]
    cur.execute("INSERT INTO actions VALUES(?, ?, ?, ?, ?, ?, ?, ?)", row)
    con.commit()
    
    # return success/fail message
    # TODO change to log
    print(f'Success: {id} updated from feed')

    con.close()

In [13]:
addBill(db=db)
# SB 2979
# SB 3381
# HB 2696
# HB 2693
# SB 2980
# HB 2549
# SB 2836
# SB 3231

In [5]:
getBills(db=db)

[('2024SB2979', 'SB', '2979', '2024', 'https://www.capitol.hawaii.gov/session/measure_indiv.aspx?billtype=SB&billnumber=2979&year=2024', 'https://www.capitol.hawaii.gov/sessions/session2024/rss/SB2979.xml', None), ('2024SB3381', 'SB', '3381', '2024', 'https://www.capitol.hawaii.gov/session/measure_indiv.aspx?billtype=SB&billnumber=3381&year=2024', 'https://www.capitol.hawaii.gov/sessions/session2024/rss/SB3381.xml', None), ('2024HB2696', 'HB', '2696', '2024', 'https://www.capitol.hawaii.gov/session/measure_indiv.aspx?billtype=HB&billnumber=2696&year=2024', 'https://www.capitol.hawaii.gov/sessions/session2024/rss/HB2696.xml', None), ('2024HB2693', 'HB', '2693', '2024', 'https://www.capitol.hawaii.gov/session/measure_indiv.aspx?billtype=HB&billnumber=2693&year=2024', 'https://www.capitol.hawaii.gov/sessions/session2024/rss/HB2693.xml', None), ('2024SB2980', 'SB', '2980', '2024', 'https://www.capitol.hawaii.gov/session/measure_indiv.aspx?billtype=SB&billnumber=2980&year=2024', 'https://ww

In [7]:
getHearings(db=db)

[]


In [6]:
checkAllUpdates(db=db)

Success: 2024SB2979 updated from feed
Success: 2024SB3381 updated from feed
Success: 2024HB2696 updated from feed
Success: 2024HB2693 updated from feed
Success: 2024SB2980 updated from feed
Success: 2024HB2549 updated from feed
Success: 2024SB2836 updated from feed
Success: 2024SB3231 updated from feed
Success: all bills updated from feed


In [23]:
# Senate e.g. for testing: 
# '2/12/24 SB2979: The committee(s) on GVO/LBT has scheduled a public hearing on 02-15-24 3:10PM; Conference Room 225 & Videoconference.'

['GVO/LBT', '02-15-24', '3:10PM', 'Conference Room 225 & Videoconference']


In [19]:
# House e.g. for testing:
#  "H 2/9/2024: Bill scheduled to be heard by JHA on Tuesday, 02-13-24 2:00PM in House conference room 325 VIA VIDEOCONFERENCE."

['JHA', '02-13-24', '2:00PM', 'House conference room 325 VIA VIDEOCONFERENCE']
